In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
%matplotlib inline

In [2]:
data = pd.read_csv("binary.csv")

In [3]:
target = data["admit"]
x_inputs = data[["gre","gpa"]]

In [4]:
weights = np.random.normal(size=3)
weights

array([ 0.88227829,  0.67088658, -0.20986808])

In [15]:
def σ(instance,weights):
    input_w_bias = np.append(1, instance)
    return 1/(1+np.exp(-np.dot(weights,input_w_bias)))

In [16]:
rand_instance = x_inputs.iloc[np.random.randint(len(data))]
σ(rand_instance,weights)

1.0

In [17]:
def cost_function(data,weights):
    target = data["admit"]
    x_inputs = data[["gre","gpa"]]
    m = len(data)
    return -1/m*sum([target[x]*np.log(σ(x_inputs.iloc[x],weights))+
                     (1-target[x])*np.log(1-σ(x_inputs.iloc[x],weights))
                     for x in target])

In [18]:
cost_function(data,weights)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in multiply
  import sys


nan

In [19]:
(1-target[1])*np.log10(1-σ(x_inputs.iloc[1],weights))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log10
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in multiply
  """Entry point for launching an IPython kernel.


nan

In [10]:
def gradient_L(data,weights):
    m = len(data)
    target = data["admit"]
    x_inputs = data[["gre","gpa"]]
    w1 = sum([σ(x_inputs.iloc[x],weights)-target[x]*weights[0] for x in target])*1/m
    w2 = sum([σ(x_inputs.iloc[x],weights)-target[x]*weights[1] for x in target])*1/m
    w3 = sum([σ(x_inputs.iloc[x],weights)-target[x]*weights[2] for x in target])*1/m
    return np.array([w1,w2,w3])

In [11]:
gradient_L(data,weights)

array([0.71987664, 0.78699351, 1.06663312])

In [12]:
cost_history = []
α = 0.125
m = len(data)
i = 1
while i < 100 and cost_function(data,weights) < 0.001:
    weights -= α*gradient_L(data,weights)
    cost_history.append(tuple((cost_function(data,weights),i)))
    i += 1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in multiply
  import sys


In [13]:
weights

array([ 0.88227829,  0.67088658, -0.20986808])

In [14]:
cost_history

[]